In [ ]:
# load data with intake

In [ ]:
# print dask dataframe

In [ ]:
# load a slice

In [ ]:
# gage map with tooltip

# demonstrate basic hvplot and dask functionality. See intro demos for either library and then cook up examples using our data that:
# - demonstrate time and spatial slicing
# - use dask to compute a mean
# - use groupby to compute a mean for each gage
# - plot a hydrograph

In [ ]:
# possible exercises
# - download gages II attributes and use to compute statistics for different regions
# - compute statistics for different seasons (see dscore example)
